In [ ]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

In [ ]:
val spark = sparkSession

In [ ]:
val training = spark.createDataFrame(Seq(
  (0L, "a b c d e spark", 1.0),
  (1L, "b d", 0.0),
  (2L, "spark f g h", 1.0),
  (3L, "hadoop mapreduce", 0.0)
)).toDF("id", "text", "label")

In [ ]:
val tokenizer = new Tokenizer()
  .setInputCol("text")
  .setOutputCol("words")

val hashingTF = new HashingTF()
  .setNumFeatures(1000)
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("features")

In [ ]:
val lr = new LogisticRegression()
  .setMaxIter(10)
  .setRegParam(0.001)

In [ ]:
val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, lr))

In [ ]:
val model = pipeline.fit(training)

In [ ]:
model.write.overwrite().save("/tmp/spark-logistic-regression-model")

In [ ]:
pipeline.write.overwrite().save("/tmp/unfit-lr-model")

In [ ]:
val sameModel = PipelineModel.load("/tmp/spark-logistic-regression-model")

In [ ]:
val test = spark.createDataFrame(Seq(
  (4L, "spark i j k"),
  (5L, "l m n"),
  (6L, "spark hadoop spark"),
  (7L, "apache hadoop")
)).toDF("id", "text")


In [ ]:
model.transform(test)
  .select("id", "text", "probability", "prediction")
  .collect()
  .foreach { case Row(id: Long, text: String, prob: Vector, prediction: Double) =>
    println(s"($id, $text) --> prob=$prob, prediction=$prediction")
  }